In [ ]:
from models.regex import RegexModel
from data_loader import AnnotationDataLoader
from model_evaluation import ModelEvalWrapper, ModelEvaluator, ModelEvalWrapperInterface, ModelReport

regex_m = RegexModel()
dataloader = AnnotationDataLoader()


In [12]:
reg_report = ModelReport.load_report("report_regex.rep")
emb_report = ModelReport.load_report("embbeder_report.rep")

In [39]:
reg_report.getattr

AttributeError: 'ModelReport' object has no attribute 'getattr'

In [ ]:
import pandas as pd
import plotly.express as px


reports = [reg_report, emb_report]
property_to_compare = "accuracy"

accuracy_dict = {r.model_title:getattr(r, property_to_compare) for r in reports}
accuracy_df = pd.DataFrame.from_dict(accuracy_dict, orient="index", columns=[property_to_compare])
accuracy_df = accuracy_df.sort_values(by=property_to_compare, ascending=False)


# fig = px.imshow(accuracy_df, text_auto=True, color_continuous_scale='Viridis')
# fig.update_layout(title="Interactive Heatmap")
# fig.show()


In [68]:
reg_report.macro_avg

{'precision': 0.9556930135115426,
 'recall': 0.6874597600234124,
 'f1-score': 0.7728902792257779,
 'support': 3500.0}

In [ ]:
# # self.metrics = None
# self.metric_per_label = None
# self.weighted_avg = None

# # memory profling 
# self.memory_records = None

self = reg_report

report_dict = {"Model" : self.model_title, 
               "Accuracy": self.accuracy,
               "Average Time Per Prediction": self.avg_time_per_prediction
}

for macro_avg_item in self.macro_avg.keys():
    report_dict[f"Macro Average ({macro_avg_item})"] =  self.macro_avg[macro_avg_item]

for weighted_avg_item in self.weighted_avg.keys():
    report_dict[f"Weighted Average ({weighted_avg_item})"] =  self.weighted_avg[weighted_avg_item]


report_dict

{'Model': 'Regex',
 'Accuracy': 0.9348571428571428,
 'Average Time Per Prediction': 0.00013082488571360175,
 'Macro Average (precision)': 0.9556930135115426,
 'Macro Average (recall)': 0.6874597600234124,
 'Macro Average (f1-score)': 0.7728902792257779,
 'Macro Average (support)': 3500.0}

In [81]:
self.weighted_avg

{'precision': 0.938310843140709,
 'recall': 0.9348571428571428,
 'f1-score': 0.9242453171724997,
 'support': 3500.0}

In [ ]:
from dash import Dash, dash_table, Input, Output, dcc, html
import pandas as pd
import numpy as np

# Example DataFrame: each row = a model, columns = metrics
models = ["Model A", "Model B", "Model C", "Model D"]
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]
data = np.random.rand(len(models), len(metrics))  # random metrics

df = pd.DataFrame(data, columns=metrics)
df.insert(0, "Model", models)  # first column = model names
df.insert(0, "Row", range(1, len(df) + 1))  # row numbers

# Choose default metrics to show
default_metrics = ["Accuracy", "F1 Score"]

# Initialize Dash app
app = Dash(__name__)

# Layout with styled checklist and default selected columns
app.layout = html.Div([
    html.Div([
        html.H4(
            "Select Metrics to Display:",
            style={'textAlign': 'center', 'fontFamily': 'Arial, sans-serif', 'marginBottom': '8px'}
        ),
        dcc.Checklist(
            id='column-selector',
            options=[{"label": col, "value": col} for col in df.columns if col not in ["Row", "Model"]],
            value=default_metrics,  # only certain columns selected by default
            inline=True,
            inputStyle={"transform": "scale(1.5)", "margin-right": "6px", "margin-left": "10px"},
            labelStyle={
                "margin-right": "20px",
                'fontFamily': 'Arial, sans-serif',
                'fontSize': '14px',
                'fontWeight': 'bold',
                'color': '#333'
            }
        )
    ], style={
        'textAlign': 'center',
        'padding': '10px',
        'backgroundColor': '#f9f9f9',
        'borderRadius': '8px',
        'marginBottom': '15px',
        'display': 'inline-block'
    }),
    
    dash_table.DataTable(
        id='model-metrics-table',
        data=df.to_dict('records'),
        sort_action="native",
        sort_mode="single",
        style_table={'width': 'auto', 'height': '400px', 'overflowX': 'auto', 'overflowY': 'auto', 'margin': 'auto'},
        style_cell={
            'textAlign': 'center',
            'padding': '4px',
            'minWidth': '40px',
            'width': 'auto',
            'maxWidth': '150px',
            'whiteSpace': 'normal',
            'fontSize': '13px',
            'lineHeight': '15px',
            'fontFamily': 'Arial, sans-serif'
        },
        style_header={
            'backgroundColor': '#f0f0f0',
            'fontWeight': 'bold',
            'textAlign': 'center',
            'fontSize': '14px',
            'height': '30px',
            'fontFamily': 'Arial, sans-serif'
        },
        style_header_conditional=[]
    )
])

# Callback to update table columns dynamically and highlight sorted header
@app.callback(
    Output('model-metrics-table', 'columns'),
    Output('model-metrics-table', 'style_header_conditional'),
    Input('model-metrics-table', 'sort_by'),
    Input('column-selector', 'value')
)
def update_table_columns(sort_by, selected_metrics):
    displayed_columns = ["Row", "Model"] + selected_metrics
    new_columns = []
    style_header_conditional = []

    for col in displayed_columns:
        col_name = col
        if sort_by and col == sort_by[0]['column_id']:
            direction = sort_by[0]['direction']
            col_name = f"{col} ({direction})"
            color = '#d1e7dd' if direction == 'asc' else '#f8d7da'
            style_header_conditional.append({
                'if': {'column_id': col},
                'backgroundColor': color
            })
        new_columns.append({"name": col_name, "id": col})

    return new_columns, style_header_conditional

# Run inline in Jupyter
app.run(mode='inline')


In [67]:
from dash import Dash, dash_table, Input, Output
import pandas as pd
import numpy as np

# Example DataFrame: each row = a model, columns = metrics
models = ["Model A", "Model B", "Model C", "Model D"]
metrics = ["Accuracy", "Precision", "Recall", "F1 Score", "asd", "Asdasdas"]
data = np.random.rand(len(models), len(metrics))  # random metrics for illustration

df = pd.DataFrame(data, columns=metrics)
df.insert(0, "Model", models)  # first column = model names

# Add row numbers if desired
df.insert(0, "Row", range(1, len(df) + 1))

# Initialize Dash app
app = Dash(__name__)

# Define columns
columns = [{"name": col, "id": col} for col in df.columns]

# Layout
app.layout = dash_table.DataTable(
    id='model-metrics-table',
    columns=columns,
    data=df.to_dict('records'),
    sort_action="native",
    sort_mode="single",
    
    # Table size
    style_table={'width': '700px', 'height': '400px', 'overflowX': 'auto', 'overflowY': 'auto', 'margin': 'auto'},
    
    # Cell style
    style_cell={
        'textAlign': 'center',
        'padding': '4px',
        'minWidth': '40px',
        'width': 'auto',
        'maxWidth': '150px',
        'whiteSpace': 'normal',
        'fontSize': '13px',
        'lineHeight': '15px'
    },
    
    # Header style
    style_header={
        'backgroundColor': '#f0f0f0',
        'fontWeight': 'bold',
        'textAlign': 'center',
        'fontSize': '14px',
        'height': '30px'
    },
    
    style_header_conditional=[]
)

# Callback to update header highlight and add (asc)/(desc)
@app.callback(
    Output('model-metrics-table', 'columns'),
    Output('model-metrics-table', 'style_header_conditional'),
    Input('model-metrics-table', 'sort_by')
)
def update_header(sort_by):
    new_columns = []
    style_header_conditional = []

    for col in df.columns:
        col_name = col
        if sort_by and col == sort_by[0]['column_id']:
            direction = sort_by[0]['direction']
            col_name = f"{col} ({direction})"
            color = '#d1e7dd' if direction == 'asc' else '#f8d7da'
            style_header_conditional.append({
                'if': {'column_id': col},
                'backgroundColor': color
            })
        new_columns.append({"name": col_name, "id": col})
    
    return new_columns, style_header_conditional

# Run inline in Jupyter
app.run(mode='inline')


In [10]:
# ## Evalute model
# regex_m_wrapper = ModelEvalWrapper(regex_m, "Regex", "The baseline regex classifer")
# m = ModelEvaluator(regex_m_wrapper, dataloader.get_train())

# c_report = m.generate_report()
# c_report.show_report()
# c_report.save_to_file("report_regex.rep")

In [ ]:
from models.pretrained_embedder.embedder_classifier import EmbedderClassifier
from data_loader import AnnotationDataLoader
from model_evaluation import ModelEvalWrapper, ModelEvaluator, ModelEvalWrapperInterface

# embedder_m = EmbedderClassifier()
# dataloader = AnnotationDataLoader()

# embedder_m.train(dataloader.get_train())

# eval_wrapper =  ModelEvalWrapper(embedder_m, "embedder")
# m = ModelEvaluator(eval_wrapper, dataloader.get_validation())
# c_report = m.generate_report()
# c_report.show_report()



In [8]:
c_report.save_to_file("embbeder_report")

'{"py/object": "model_evaluation.model_evaluator.ModelReport", "model_title": "embedder", "model_info": null, "metric_per_label": {"Uninformative": {"precision": 0.8904109589041096, "recall": 0.7471264367816092, "f1-score": 0.8125, "support": 87.0}, "Low": {"precision": 1.0, "recall": 0.8717948717948718, "f1-score": 0.9315068493150684, "support": 39.0}, "Proper": {"precision": 0.9720044792833147, "recall": 0.9931350114416476, "f1-score": 0.9824561403508771, "support": 874.0}}, "accuracy": 0.967, "macro_avg": {"precision": 0.9541384793958082, "recall": 0.870685440006043, "f1-score": 0.9088209965553151, "support": 1000.0}, "weighted_avg": {"precision": 0.9659976683182746, "recall": 0.967, "f1-score": 0.9656829337899543, "support": 1000.0}, "cm": {"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3, 3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tupl

In [19]:
### make the relative imports work in jp
##
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
##
import os
import pandas as pd
from math import isclose
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
